In [1]:
import pyvips
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="3"
import cupy
import matplotlib.pyplot as plt
from cuml.cluster import KMeans
import cupy 
import cv2
import IPython


Images need to be preprocessed

In [ ]:
p1="tonsil_resize_gray_dapi.tif"
p2="tonsil_resize_gray_he.tif"

In [2]:
import numpy as np

format_to_dtype = {
    'uchar': np.uint8,
    'char': np.int8,
    'ushort': np.uint16,
    'short': np.int16,
    'uint': np.uint32,
    'int': np.int32,
    'float': np.float32,
    'double': np.float64,
    'complex': np.complex64,
    'dpcomplex': np.complex128,
}

img = pyvips.Image.new_from_file(p1, access='sequential')

In [4]:
img3

<pyvips.Image 9444x7576 uchar, 3 bands, srgb>

In [5]:
max_data=11*1024**3
data_to_use_init=img3.height*img3.width*img3.bands*8*6
data_to_use=(512+data_to_use_init-data_to_use_init%512)

In [6]:
data_to_use/512

20122804.0

In [7]:
max_data

11811160064

In [8]:
if(data_to_use>max_data):
    print("Don't!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n Shutting Down for safety ")
    IPython.Application.instance().kernel.do_shutdown(True)
else:
    print("Ok to proceed, data to use {}\n Amount of memory left = {}".format(data_to_use/(1024**3),
                                                                              (max_data-data_to_use)/(1024**3)))

Ok to proceed, data to use 9.595300674438477
 Amount of memory left = 1.4046993255615234


In [9]:
mempool = cupy.get_default_memory_pool()
pinned_mempool = cupy.get_default_pinned_memory_pool()


mempool.free_all_blocks()
mempool.set_limit(size=max_data)
print(mempool.used_bytes())          
print(mempool.total_bytes()-mempool.free_bytes())


0
0


In [10]:
img3

<pyvips.Image 9444x7576 uchar, 3 bands, srgb>

In [11]:
img_plt=np.ndarray(buffer=img3.write_to_memory(),
                   dtype=format_to_dtype[img3.format],
                   shape=[img3.height,img3.width, img3.bands])
#plt.figure(figsize = (20,20))
#plt.imshow(img_plt/255)

In [12]:
img_plt_cp=cupy.asarray(img_plt,dtype="float64")
img_plt_cp.shape

(7576, 9444, 3)

In [13]:
b = img_plt_cp.reshape(-1,3)/255

In [15]:
clusters=4
#print("input:")
#print(b)

print("Calling fit")
kmeans_float = KMeans(n_clusters=clusters)
kmeans_float.fit(b)

print("labels:")
print(kmeans_float.labels_)
print("cluster_centers:")
print(kmeans_float.cluster_centers_)

Calling fit
labels:
[2 2 2 ... 2 2 2]
cluster_centers:
[[0.67980726 0.49370966 0.7161153 ]
 [0.35369195 0.25683873 0.57062674]
 [0.82081788 0.81540177 0.79757177]
 [0.51497114 0.37001306 0.64761655]]


In [16]:
labels=kmeans_float.labels_

In [17]:
labels.shape

(71547744,)

In [18]:
print(mempool.used_bytes())          
print(mempool.total_bytes()-mempool.free_bytes())

0
0


In [19]:
path="Trial_020821"
dtype_to_format = {
    'uint8': 'uchar',
    'int8': 'char',
    'uint16': 'ushort',
    'int16': 'short',
    'uint32': 'uint',
    'int32': 'int',
    'float32': 'float',
    'float64': 'double',
    'complex64': 'complex',
    'complex128': 'dpcomplex',
}
os.makedirs(path,exist_ok=True)



In [22]:
x=0

In [30]:
try:
    del temp
    del img_plt
    del vi
except:
    print("Some or more variables do not exist")

In [34]:

    
img = pyvips.Image.new_from_file('resized_Tonsil_HE.tif', access='sequential')
img3=img.resize(0.2)
img_plt=np.ndarray(buffer=img3.write_to_memory(),
                   dtype=format_to_dtype[img3.format],
                   shape=[img3.height,img3.width, img3.bands])
#plt.figure(figsize = (20,20))
img_plt_cp=cupy.asarray(img_plt,dtype="float64")
img_plt_cp.shape
temp = img_plt_cp.reshape(-1,3)/255

#plt.imshow(img_plt/255)


In [35]:
img_plt_cp.shape[1]*img_plt_cp.shape[0]/temp.shape[0]

1.0

In [36]:
print(x)
temp[labels!=x]=[0,0,0]
vi = pyvips.Image.new_from_memory(cupy.asnumpy(temp), img_plt_cp.shape[1], img_plt_cp.shape[0], 3,
                                  'double')
vi.tiffsave(path+"/"+f"{x:03d}"+".tif")
x=x+1

3
